
# Emotion Detection Model Analysis

This notebook provides a comprehensive analysis of the Emotion Detection Model. The analysis includes loading a pre-trained model, processing video data frame by frame to predict emotions, and saving the results which consist of emotion and confidence level per frame.

## Table of Contents
1. Load Pre-trained Model
2. Video Processing and Emotion Prediction
3. Saving the Results
    

In [2]:
from keras.models import load_model
import cv2
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.patches as patches

## Load the model

run 
```python
    model.summary()
```
to see the model architecture.


In [4]:
model_name = 'Riley_12' #This is the name of the model you want to load, other option is 'Riley_4'
model = load_model(f'models/{model_name}.h5', compile=False)

In [5]:
class_labels = ['angry', 'disgust', 'fear', 'happy', 'neutral', 'sad', 'surprise']

In [6]:
def preprocess_image(frame, target_size):
    img = cv2.resize(frame, target_size)  # Resize to 64x64 pixels
    img = img / 255.0  # Normalize pixel values to [0, 1]
    img = np.expand_dims(img, axis=-1)  # Add channel dimension for grayscale
    img = np.expand_dims(img, axis=0)  # Add batch dimension
    return img

# Function to predict emotion for a frame
def predict_emotion(frame):
    preprocessed_frame = preprocess_image(frame, (48, 48))
    prediction = model.predict(preprocessed_frame)
    predicted_class_index = np.argmax(prediction)
    predicted_class = class_labels[predicted_class_index]
    confidence_level = prediction[0][predicted_class_index]
    return predicted_class, confidence_level, prediction

In [7]:
person = 'person3' #This is the name of the person you want to test, other options are 'person1' and 'person2'
video_path = f"data/clase_{person}.mp4" #This is the path to the video you want to test
cap = cv2.VideoCapture(video_path)

if not cap.isOpened():
    print("Error opening video stream or file")


## Video Processing and Emotion Prediction

Here, we import the video and process it frame by frame. For each frame, the model predicts the emotion displayed on the face. The predicted emotion and its confidence level are stored for further analysis.
            

In [8]:
frame_interval = 6

data = pd.DataFrame(columns=['frame', 'emotion', 'confidence'])
predictions = []
frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    
    # Process every n-th frame based on the frame_interval
    if frame_count % frame_interval == 0:
        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
        predicted_class, confidence_level, prediction = predict_emotion(gray_frame)
        data = pd.concat([data, pd.DataFrame({'frame': frame_count, 'emotion': predicted_class, 'confidence': confidence_level}, index=[0])], ignore_index=True)
        predictions.append(predictions)
        print(f'Frame {frame_count}: {predicted_class} ({confidence_level:.2f})')

    frame_count += 1

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 173ms/step
Frame 0: neutral (0.62)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Frame 6: neutral (0.45)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Frame 12: neutral (0.48)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Frame 18: neutral (0.37)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Frame 24: fear (0.30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Frame 30: neutral (0.44)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Frame 36: fear (0.30)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Frame 42: neutral (0.70)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Frame 48: neutral (0.60)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step
Frame 54: neutral (0.65)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Frame 60: neutral (0.54)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Frame 66: neutral (0.64)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Frame 72: neutral (0.57)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
Frame 78: neutral (0.60)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
Frame 84: neutral (0.74)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 20ms/step
Frame 90: neutral (0.79)

In [318]:
data.to_csv(f'outputdata/{person}_{model_name}.csv', index=False)